In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import os
import pickle
import joblib
import plotly.graph_objects as go
import re
pd.set_option('display.max_columns', None)

In [19]:
pip install pytorch


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pytorch
Failed to build pytorch


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\guitz\AppData\Local\Temp\pip-install-xnawvm6x\pytorch_8736d287e8ef43f8b704b4799eab8404\setup.py", line 15, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects


In [11]:
class CandleFit:
    def __init__(self, data_path: pd.DataFrame = None, ticker: str = None, timeframe: str = '5y'):
        """
        Initializes the CandleFit object with a specified ticker and timeframe.

        Parameters:
        ticker (str): The ticker symbol of the stock.
        data_path (str): The absolute path to a CSV file or DataFrame containing HLOC and volume data. 
            The file must have columns indexed by date and time for each transaction.
            Example csv format:
                Date,        Time,    Open,    High,    Low     Close   Volume
                2024.08.28,  00:00,   1.1176,  1.11838, 1.1176,  1.11822, 167,
                2024.08.27,  23:00,   1.1184,  1.11855, 1.11824, 1.11835, 449,
                2024.08.27,  22:00,   1.11855, 1.11855, 1.11811, 1.11839, 918,
                2024.08.27,  21:00,   1.1181,  1.11902, 1.11786, 1.11855, 1762,

        timeframe (str): The timeframe for which to download historical data and to load from data_path.
        """
        self.ticker = ticker
        self.data_path = data_path
        self.timeframe = timeframe
        self.data = self.get_data()
        self.features = self.get_price_features()
        self.threshold_dict: dict = None
    
    def get_freq(self):
        """
        Retrieves the resampling frequency corresponding to the timeframe defined in the `self.timeframe` attribute.

        The method uses a mapping between MetaTrader timeframes and Pandas resampling frequencies to determine the correct frequency.

        Returns:
            str: The resampling frequency as a string in a format accepted by Pandas,
                or `None` if the timeframe is not mapped.
        """
        timeframe_mapping = {
            'M1': '1min', 'M5': '5min', 'M15': '15min', 'M30': '30min',
            'H1': '1H', 'H4': '4H', 'D1': '1D', 'W1': '1W', 'MN': '1M'
        }
        resample_freq = timeframe_mapping.get(self.timeframe.upper())
        return resample_freq

    
    def get_data(self):
        """
        Downloads historical data for the specified ticker and timeframe or loads a previous file containing HLOC and volume.

        Returns:
        pd.DataFrame: A DataFrame containing the historical data with a combined datetime index.
        """
        
        if self.ticker is not None:
            try:
                ticker_obj = yf.Ticker(self.ticker)
                data = ticker_obj.history(interval=self.timeframe.lower())
                data.index = pd.to_datetime(data.index).strftime('%Y-%m-%d %H:%M:%S')
                data.columns = [col.lower() for col in data.columns]
                return data
            except Exception as e:
                display(f"Error downloading ticker data: {e}")
                return pd.DataFrame()
        else:
            valid_timeframes = ['M1', 'M5', 'M15', 'M30', 'H1', 'H4', 'D1', 'W1', 'MN']
            if self.timeframe.upper() not in valid_timeframes:
                display(f"Invalid timeframe. Please select one of the following: {valid_timeframes}")
                return pd.DataFrame()

            if self.data_path is not None:
                file_path = self.data_path  
                file_ext = os.path.splitext(file_path)[-1].lower()
                try:
                    if file_ext == ".csv":
                        try:
                            df = pd.read_csv(file_path, encoding='utf-16', delimiter=',')
                        except UnicodeDecodeError:
                            df = pd.read_csv(file_path, encoding='utf-8', delimiter=',')
                        
                        df.columns = [col.lower() for col in df.columns]
                        
                        if df.columns[0] == 'date':
                            df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str), format='%Y.%m.%d %H:%M')
                            df.drop(columns=['date', 'time'], inplace=True)
                        else:
                            display("Unexpected format in CSV file.")
                            return pd.DataFrame()

                    elif file_ext == ".xlsm":
                        df = pd.read_excel(file_path, engine='openpyxl', parse_dates=[['Date', 'Time']])
                        df.columns = [col.lower() for col in df.columns]
                    elif file_ext == ".parquet":
                        df = pd.read_parquet(file_path)
                        if 'Date_Time' in df.columns:
                            df['datetime'] = pd.to_datetime(df['Date_Time'])
                            df.drop(columns=['Date_Time'], inplace=True)
                        df.columns = [col.lower() for col in df.columns]
                    else:
                        display("Unsupported file type.")
                        return pd.DataFrame()
                except Exception as e:
                    display(f"Error loading market data: {e}")
                    return pd.DataFrame()
            else:
                display("No market data or ticker specified.")
                return pd.DataFrame()

            if 'datetime' not in df.columns:
                display("Column 'Datetime' is missing from the DataFrame.")
                return pd.DataFrame()

            resample_freq = self.get_freq()
            
            df.set_index('datetime', inplace=True)
            data = df.resample(resample_freq).agg({
                'open': 'first',
                'high': 'max',
                'low': 'min',
                'close': 'last',
                'volume': 'sum'
            }).dropna()

        return data

    
    @staticmethod
    def load_dict(key: str) -> dict:
        """
        Loads a dictionary from a pickle file based on the provided key.

        Parameters:
        key (str): The key to search for in the dictionary.

        Returns:
        dict: The dictionary associated with the provided key.

        Raises:
        FileNotFoundError: If the pickle file is not found.
        KeyError: If the key is not found in the dictionary.
        ValueError: If there is an error loading the pickle file.
        """
        filepath = os.path.join('..', 'pkl', 'threshold_dicts.pkl')
        try:
            with open(filepath, 'rb') as file:
                data = pickle.load(file)
            
            for item in data:
                if key in item:
                    return item[key]
            raise KeyError(f"Key '{key}' not found in the threshold dictionary.")
        except FileNotFoundError:
            raise FileNotFoundError(f"The file '{filepath}' was not found.")
        except pickle.PickleError:
            raise ValueError("Error occurred while loading the pickle file.")
        
    def get_price_features(self):
        """
        Calculates various price and volume features from dataorical data.

        Returns:
        pd.DataFrame: A DataFrame containing the calculated features.
        """
        aux = self.data.copy()
        df = pd.DataFrame(index=aux.index)
        df.index = pd.to_datetime(df.index)
        df['return_rate'] = aux['close'].pct_change()
        df['volume_change'] = aux['volume'].diff()
        df['volume_var'] = aux['volume'].pct_change() + 1
        df['price_range'] = aux['high'] - aux['low']
        df['price_var'] = df['price_range'] / aux['low']
        df['price_change'] = aux['close'] - aux['open']
        df['close_vol'] = aux['close'].expanding().std()
        df['low_vol'] = aux['low'].expanding().std()
        df['high_vol'] = aux['high'].expanding().std()
        df['open_vol'] = aux['open'].expanding().std()
        df['upper_wick'] = aux['high'] - aux[['open', 'close']].max(axis=1)
        df['lower_wick'] = aux[['open', 'close']].min(axis=1) - aux['low']
        df['wick_change'] = df['upper_wick'] - df['lower_wick']
        df['wick_var'] = df['wick_change'] / df['lower_wick']
        df['std_wick'] = df['wick_change'].abs().expanding().std()
        df = df.apply(pd.to_numeric, errors='coerce')
        df = pd.concat([aux, df], axis=1)
        
        return df

    def get_candle_features(self, 
                            doji_threshold: float =  0.002, 
                            bullish_threshold : float =  0.005,
                            bearish_threshold: float =  0.005,
                            volatility_window: int = 7):
                            
 
        """
        Calculates various candlestick features based on price data and a threshold dictionary.

        Parameters:
        threshold_dict (dict): A dictionary containing thresholds for calculating features. If None, it loads the default dictionary.

        Returns:
        pd.DataFrame: A DataFrame containing the calculated candlestick features.
        """
        
        aux = self.features.copy()
        aux = aux.loc[:, ~aux.columns.duplicated()]
        df = pd.DataFrame(index=aux.index)
        df[f'std_volatility_window'] = aux['price_change'].rolling(window=volatility_window).std().abs()
        df['bearish_threshold'] = pd.to_numeric(bearish_threshold * df[f'std_volatility_window'], errors='coerce').fillna(0)
        df['bullish_threshold'] = pd.to_numeric(bullish_threshold * df[f'std_volatility_window'], errors='coerce').fillna(0)
        df['is_bearish'] = (aux['close'] <= (aux['open'] - df['bearish_threshold'])).astype(int)
        df['is_bullish'] = (aux['close'] >= (aux['open'] + df['bullish_threshold'])).astype(int)
        df['is_doji'] = (abs(aux['close'] - aux['open']) <= doji_threshold).astype(int)
        df['is_bearish_open_gap'] = (aux['open'] < aux['close'].shift(1)).astype(int)
        df['is_bullish_open_gap'] = (aux['open'] > aux['close'].shift(1)).astype(int)
    
        df = pd.concat([aux, df], axis=1)
        self.features = df
        return  self.features

    def fit_morning_star(self,
                         doji_threshold: float = 0.002,
                         bullish_to_bearish_ratio : float = 0.33,
                         bearish_threshold: float = 0.2):
                  
        """
        Identifies the morning star candlestick pattern in the dataorical data.

        Parameters:
        doji_threshold (float): The threshold for the price change to identify a doji candle. Default is 0.002.
        bullish_threshold (float): The ratio of the price change on the third day to the price change on the first day to identify a bullish candle. Default is 0.33.
        bearish_threshold (float): The threshold to identify a bearish candle on the first day. Default is 0.2.
        volatility_window (int): The window size (in days) for calculating the volatility of the dataorical data. Default is 7.

        Returns:
        pd.DataFrame: A DataFrame with a column indicating the presence of the morning star pattern.
        """
        df = self.get_candle_features()  
        df = df.loc[:, ~df.columns.duplicated(keep='last')]
        

        # Condition 1: Two days ago was a bearish candle and the close of that day is lower than the open of that day adjusted by the threshold
        df['is_bearish_morning_star'] = ((df['is_bearish'].shift(2) == 1) & 
                                        (df['close'].shift(2) + bearish_threshold * df['price_change'].shift(2) 
                                        <= df['open'].shift(2))).astype(int)

        # Condition 2: The previous day was a doji candle and had a bearish open gap
        df['is_bearish_open_gap_morning_star'] = (df['is_bearish_open_gap'].shift(1) == 1).astype(int)
        df['is_doji_morning_star'] = (df['price_change'].shift(1).abs() <= doji_threshold ).astype(int)

        # Condition 3: Today is a bullish candle and the price change from two days ago to today is significant
        df['is_bullish_morning_star'] = ((df['is_bullish'] == 1) & \
                                        (df['close'] - df['close'].shift(2) >= bullish_to_bearish_ratio)
                                        * df['price_change'].shift(2).abs()).astype(int)

        # Combine all conditions to determine the morning star pattern
        df['is_morning_star'] = df[['is_bearish_morning_star', 'is_bearish_open_gap_morning_star', 'is_doji_morning_star', 
                                    'is_bullish_morning_star']].all(axis=1).astype(int)

        self.features = df
        return self.features
    
    
    def fit_hammer(self,
                handle_treshold: float = 7,
                head_treshold: float = 0.1,
                bald_treshold: float = 0.005,
                volatility_window: int = 7):

        df = self.get_candle_features(volatility_window)
        df = df.loc[:, ~df.columns.duplicated(keep='last')]

        df['is_hammer_head'] = ((df['price_change'].abs() <= head_treshold * df[['close', 'open']].mean(axis=1)) & \
                                (df['price_change'].abs() >= df['std_volatility_window'])).astype(int)
                
        df['is_hammer_handle'] = (df['lower_wick'] >= df['price_change'].abs() * 
                                  (1 + handle_treshold)*df['std_volatility_window']).astype(int)

        df['is_hammer_bald'] = (df['upper_wick'] <= (1 + bald_treshold) * df[['open', 'close']].max(axis=1)).astype(int)

        df['is_hammer'] = df[['is_hammer_head', 'is_hammer_handle', 'is_hammer_bald']].all(axis=1).astype(int)

        self.features = df
        return self.features


    
    def get_movings(self, short:int = None, long:int = None, strategy: str = 'test'):

        """
        Calculates buy and sell signals based on moving averages for different strategies and parameters.

        Parameters:
        - threshold_dict (dict, optional): Dictionary containing moving average settings for different strategies. If any of parameters is None,
        it will be loaded with `self.load_dict(key='rolling_cross_dict')` and available strategies will be measured.  
        - short (int, optional): Time window for the short moving average. Not used directly in the function.
        - long_ (int, optional): Time window for the long moving average. Not used directly in the function.
        - signal_window (int, optional): Time window for signal calculation. Not used directly in the function.

        Returns:
        - pd.DataFrame: DataFrame with additional columns for moving averages and buy/sell signals.
        """
        df = self.get_candle_features()  
        df = df.loc[:, ~df.columns.duplicated(keep='last')]
        
        if (short is None) != (long is None):
            raise ValueError("Please set both short and long to valid int or set both to None.")
        elif all(x is not None for x in (short, long)):
            threshold_dict = {f'{strategy}_{short}_{long}': {'short': short, 'long': long}}
        else:
            display('Loading standard strategies')
            threshold_dict = self.load_dict(key='rolling_cross_dict')

        tolerance = 0.015 * df['close_vol']  
        for key, value in threshold_dict.items():
            strategy = key.split('_')[0] if '_' in key else key
            short = value['short']
            long = value['long']
            
            df.loc[:, f'{strategy}_short_{short}'] = df['close'].rolling(window=short, min_timeframes=1).mean()
            df.loc[:, f'{strategy}_long_{long}'] = df['close'].rolling(window=long, min_timeframes=1).mean()
            
            
            buy = (df[f'{strategy}_short_{short}'] > df[f'{strategy}_long_{long}'] + tolerance) & \
                (df[f'{strategy}_short_{short}'].shift(1) < df[f'{strategy}_long_{long}'].shift(1) + tolerance)
            
            sell = (df[f'{strategy}_short_{short}'] < df[f'{strategy}_long_{long}'] - tolerance) & \
                (df[f'{strategy}_short_{short}'].shift(1) > df[f'{strategy}_long_{long}'].shift(1) - tolerance)
            
            df.loc[:, f'{strategy}_{short}_{long}'] = np.where(buy, 1, np.where(sell, -1, 0))

                        
        self.features = df
        self.threshold_dict = threshold_dict  
        return self.features          

    def candlestick_chart(self, 
                      key: str = 'is_morning_star', 
                      plot_type: str = 'pattern',
                      height: int = 1000, 
                      offset: float = 0.1,
                      start_date: str = None,
                      finish_date: str = None):
        """
        Generates a candlestick chart with optional pattern or price action markers.

        Parameters:
        - key (str): Key for identifying the pattern or price action indicators. Defaults to 'is_morning_star'.
        - plot_type (str): Type of plot to generate. Options are 'pattern' or 'price_action'. Defaults to 'pattern'.
        - height (int): Height of the plot in pixels. Defaults to 1200.
        - offset (float): Vertical offset for pattern markers. Defaults to 0.1.
        - start_date (str): Start date for filtering the data. Format should be 'YYYY-MM-DD'.
        - finish_date (str): Finish date for filtering the data. Format should be 'YYYY-MM-DD'.

        Returns:
        - go.Figure: A Plotly Figure object with the candlestick chart.
        """
        
        aux = self.features.copy()
        aux = aux.loc[:, ~aux.columns.duplicated(keep='last')]
        
        aux.index = pd.to_datetime(aux.index)
        

        if key not in aux.columns:
            print(f"Key '{key}' not found in aux columns.")
            return None
                
        if start_date:
            start_date = pd.to_datetime(start_date)
            aux = aux[aux.index >= start_date]
        if finish_date:
            finish_date = pd.to_datetime(finish_date)
            aux = aux[aux.index <= finish_date]
        
        if aux.empty:
            print("No data available for the selected date range.")
            return None
        
        trace = go.Candlestick(
            x=aux.index,
            open=aux["open"],
            high=aux["high"],
            low=aux["low"],
            close=aux["close"],
            name=self.ticker,
            yaxis="y"
        )
        
        volume_colors = ['green' if aux['volume'][i] > aux['volume'][i-1] else 'red' for i in range(1, len(aux))]
        volume_colors.insert(0, 'green')
        volume_trace = go.Bar(
            x=aux.index,
            y=aux['volume'],
            marker_color=volume_colors,
            name='Volume',
            yaxis="y2"
        )
        
        if plot_type == 'pattern':     
            markers = aux[aux[key] == 1].copy()
            markers['close'] = markers['low'] * (1 - offset)
            markers = markers.dropna(subset=['close'])
            marker_trace = go.Scatter(
                x=markers.index,
                y=markers['close'],
                mode='markers',
                marker=dict(
                    color='blue',
                    size=8,
                    symbol='triangle-up'
                ),
                name=key,
                yaxis="y"
            )
            
            data = [trace, marker_trace, volume_trace]
            
        elif plot_type == 'moving_cross':
            strategy, short, long = key.split('_')
            short_col = f'{strategy}_short_{short}'
            long_col = f'{strategy}_long_{long}'
            signal_col = f'{strategy}_{short}_{long}'
            short_trace = go.Scatter(
                x=aux.index,
                y=aux[short_col],
                mode='lines',
                name=f'{strategy.capitalize()} Short {short}',
                line=dict(color='purple')  
            )
            long_trace = go.Scatter(
                x=aux.index,
                y=aux[long_col],
                mode='lines',
                name=f'{strategy.capitalize()} Long {long}',
                line=dict(color='orange')  
            )

            markers = aux[aux[signal_col] != 0].copy()  
            markers['close'] = markers.apply(
                lambda row: row['low'] - offset if row[signal_col] == 1 else row['high'] * (1 + offset),
                axis=1
            )
            markers['color'] = markers[signal_col].apply(lambda x: 'green' if x == 1 else 'red')
            markers['symbol'] = markers[signal_col].apply(lambda x: 'arrow-up' if x == 1 else 'arrow-down')
            
            marker_trace = go.Scatter(
                x=markers.index,
                y=markers['close'],
                mode='markers',
                marker=dict(
                    color=markers['color'],
                    size=8,
                    symbol=markers['symbol']
                ),
                name='Signal',
                yaxis="y"
            )
            
            data = [trace, short_trace, long_trace, marker_trace, volume_trace]
        
        layout = go.Layout(
            title=f"{self.ticker} Candlestick Chart markers: {key.capitalize()}",
            xaxis=dict(title="Date"),
            yaxis=dict(title="Price", domain=[0.3, 1]),
            yaxis2=dict(title="Volume", domain=[0, 0.2]),
            height=height,
            barmode='relative'
        )
        
        fig = go.Figure(data=data, layout=layout)
        return fig

                
    def get_trades(self,  
                   strategy: str = None,
                   reward_risk_ratio: list = None, 
                   price_col: str = 'close',
                   trade_timeframe=7,
                   target_return=0.01):
        """
        Calculates potential trades based on a given strategy and parameters.

        Parameters:
        df (DataFrame): The DataFrame containing price data and strategy signals.
        strategy (str): The column name of the strategy signals (1 for buy, -1 for sell).
        reward_risk_ratio (list): List of reward to risk ratios to consider.
        price_col (str): The column name to use as the price for entering trades.
        trade_timeframe (int): The number of timeframes to look ahead for the trade.
        target_return (float): The target return for each trade.

        Returns:
        DataFrame: The DataFrame with trade information, results, and positions.
        """
        
        df = self.features.copy()
        
        cols = ['open', 'high', 'low', 'close']
        shifted_cols = []
        stop_loss_cols = []
        
        for col in cols:
            for i in range(2, trade_timeframe + 1):
                df[f'{col}_{i}'] = df[col].shift(-i)
                shifted_cols.append(f'{col}_{i}')
        
        has_signals = (df[strategy] == 1) | (df[strategy] == -1)
        if not has_signals.any():
            return "The strategy did not generate buy or sell signals."
        
        df = df[cols + [strategy] + shifted_cols][has_signals].copy()
        df['side'] = df[strategy].apply(lambda x: 'long' if x == 1 else 'short')
        df['tp'] = df.apply(lambda row: (1 + target_return) * row[price_col] if row[strategy] == 1 else (1 - target_return) * row[price_col], axis=1)
        if reward_risk_ratio is None:      
            reward_risk_ratio = np.arange(0.5, 2.50, 0.5)
        
        for rr in reward_risk_ratio:
            df[f'sl_{rr}'] = np.where(df['side'] == 'long', df[price_col] * (1 - target_return * rr),df[price_col] * (1 + target_return * rr))
            stop_loss_cols.append(f'sl_{rr}')

    
        for loss_col in stop_loss_cols:
            df[f'out_day_{loss_col}'] = np.nan
            df[f'result_{loss_col}'] = np.nan
            df[f'result_{loss_col}_value'] = np.nan
            
            for index, row in df.iterrows():
                stop_loss_val = row[loss_col]
                target_price = row['tp']
                entry_price = row[price_col] 
                
                for i in range(2, trade_timeframe + 1):
                    next_open = row[f'open_{i}']
                    next_high = row[f'high_{i}']
                    next_low = row[f'low_{i}']
                    next_close = row[f'close_{i}']
                    
                    if row['side'] == 'long':
                        price_vars = [next_open, next_low, next_high, next_close]
                        for price_var in price_vars:
                            if price_var > target_price:
                                df.at[index, f'out_day_{loss_col}'] = int(i)
                                df.at[index, f'result_{loss_col}'] = 'profit'
                                df.at[index, f'result_{loss_col}_value'] = price_var - entry_price
                                break
                            elif price_var < stop_loss_val:
                                df.at[index, f'out_day_{loss_col}'] = int(i)
                                df.at[index, f'result_{loss_col}'] = 'loss'
                                df.at[index, f'result_{loss_col}_value'] = price_var - entry_price
                                break
                    else:  # 'short'
                        price_vars = [next_open, next_high, next_low, next_close]
                        for price_var in price_vars:
                            if price_var < target_price:
                                df.at[index, f'out_day_{loss_col}'] = int(i)
                                df.at[index, f'result_{loss_col}'] = 'profit'
                                df.at[index, f'result_{loss_col}_value'] = entry_price - price_var
                                break
                            elif price_var > stop_loss_val:
                                df.at[index, f'out_day_{loss_col}'] = int(i)
                                df.at[index, f'result_{loss_col}'] = 'loss'
                                df.at[index, f'result_{loss_col}_value'] = entry_price - price_var
                                break
                    
                    if not np.isnan(df.at[index, f'out_day_{loss_col}']):
                        break
        
        return df
    
    @staticmethod
    def summarize_results(df):
        """
        Summarize profit and loss results for different stop loss levels.

        This function calculates the total profit and loss for each stop loss level present
        in the DataFrame. The stop loss levels are identified dynamically based on column names
        that follow the pattern 'result_sl_X.Y'.

        Args:
            df (pd.DataFrame): The DataFrame containing the trade results. The DataFrame must
                            have columns with names in the format 'result_sl_X.Y' and
                            'result_sl_X.Y_value', where X.Y represents the stop loss level.

        Returns:
            pd.DataFrame: A DataFrame with columns 'Level', 'Profit', and 'Loss', summarizing
                        the total profit and loss for each stop loss level.
        """
        summary_df = pd.DataFrame(columns=['Risk Reward Ratio', 'Profit', 'Loss', 'Net', 'Profits Count', 'Losses Count', 'Success Rate'])
        
        levels = set()
        for col in df.columns:
            match = re.match(r'result_sl_(\d+\.\d+)', col)
            if match:
                levels.add(match.group(1))
        
        levels = sorted(levels, key=lambda x: float(x))        
        summary_list = []
        for level in levels:
            results_col = f'result_sl_{level}'
            values_col = f'result_sl_{level}_value'
            
            if results_col in df.columns and values_col in df.columns:
                df[values_col] = pd.to_numeric(df[values_col], errors='coerce')
                
                profits_series = df[df[values_col] > 0][values_col]
                losses_series = df[df[values_col] < 0][values_col]
                
                profits = profits_series.sum()
                profits_count = profits_series.count()
                
                losses = losses_series.sum()
                losses_count = losses_series.count()
                
                total_trades = profits_count + losses_count
                success_rate = profits_count / total_trades if total_trades > 0 else 0
                
                summary_list.append({
                    'Risk Reward Ratio': level,
                    'Profit': profits,
                    'Loss': losses,
                    'Net': profits + losses,
                    'Profits Count': profits_count,
                    'Losses Count': losses_count,
                    'Success Rate': success_rate
                })
        
        summary_df = pd.concat([summary_df, pd.DataFrame(summary_list)], ignore_index=True)
        
        return summary_df
    


In [12]:

ticker = CandleFit(data_path='C:\\Users\\guitz\\OneDrive\\Área de Trabalho\\pyquant\\pyquant\\csv\\prices_volume.csv', 
                   timeframe='H4')


In [13]:
(5.02810 / 5.0258) -1

0.00045763858490199816

In [14]:
5.02810
5.0304 - 5.0258

0.0045999999999999375

In [15]:

ticker.fit_hammer(volatility_window=40)


,open,high,low,close,volume,return_rate,volume_change,volume_var,price_range,price_var,price_change,close_vol,low_vol,high_vol,open_vol,upper_wick,lower_wick,wick_change,wick_var,std_wick,std_volatility_window,bearish_threshold,bullish_threshold,is_bearish,is_bullish,is_doji,is_bearish_open_gap,is_bullish_open_gap,is_hammer_head,is_hammer_handle,is_hammer_bald,is_hammer
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-09-29 16:00:00,5.0258,5.0358,5.0250,5.0304,1310,NaN,NaN,NaN,0.0108,0.002149,0.0046,NaN,NaN,NaN,NaN,0.0054,0.0008,0.0046,5.750000,NaN,NaN,0.000000,0.000000,0,1,1,0,0,0,0,1,0
2023-09-29 20:00:00,5.0304,5.0564,5.0295,5.0477,5514,0.003439,4204.0,4.209160,0.0269,0.005348,0.0173,0.012233,0.003182,0.014566,0.003253,0.0087,0.0009,0.0078,8.666667,0.002263,NaN,0.000000,0.000000,0,1,1,0,0,0,0,1,0
2023-10-02 12:00:00,5.0590,5.0731,5.0570,5.0715,1261,0.004715,-4253.0,0.228691,0.0161,0.003184,0.0125,0.020635,0.017323,0.018684,0.017988,0.0016,0.0020,-0.0004,-0.200000,0.003711,NaN,0.000000,0.000000,0,1,1,0,1,0,0,1,0
2023-10-02 16:00:00,5.0713,5.0991,5.0703,5.0924,12487,0.004121,11226.0,9.902458,0.0288,0.005680,0.0211,0.027132,0.021783,0.026772,0.022052,0.0067,0.0010,0.0057,5.700000,0.003114,NaN,0.000000,0.000000,0,1,1,1,0,0,0,1,0
2023-10-02 20:00:00,5.0922,5.0966,5.0808,5.0864,4790,-0.001178,-7697.0,0.383599,0.0158,0.003110,-0.0058,0.026197,0.024613,0.026900,0.027931,0.0044,0.0056,-0.0012,-0.214286,0.003101,NaN,0.000000,0.000000,1,0,1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26 16:00:00,5.4926,5.5194,5.4843,5.4982,11949,0.000983,9955.0,5.992477,0.0351,0.006400,0.0056,0.241602,0.238439,0.244295,0.241225,0.0212,0.0083,0.0129,1.554217,0.005652,0.033099,0.000165,0.000165,0,1,1,1,0,0,1,1,0
2024-08-26 20:00:00,5.4977,5.4978,5.4809,5.4976,5598,-0.000109,-6351.0,0.468491,0.0169,0.003083,-0.0001,0.241804,0.238640,0.244466,0.241431,0.0001,0.0167,-0.0166,-0.994012,0.005663,0.030727,0.000154,0.000154,0,0,1,1,0,0,1,1,0
2024-08-27 12:00:00,5.5003,5.5111,5.4882,5.4953,1981,-0.000418,-3617.0,0.353876,0.0229,0.004173,-0.0050,0.241999,0.238855,0.244664,0.241641,0.0108,0.0071,0.0037,0.521127,0.005660,0.028032,0.000140,0.000140,1,0,1,0,1,0,1,1,0


In [16]:


ticker.features[ticker.features['is_hammer_handle']==1]

,open,high,low,close,volume,return_rate,volume_change,volume_var,price_range,price_var,price_change,close_vol,low_vol,high_vol,open_vol,upper_wick,lower_wick,wick_change,wick_var,std_wick,std_volatility_window,bearish_threshold,bullish_threshold,is_bearish,is_bullish,is_doji,is_bearish_open_gap,is_bullish_open_gap,is_hammer_head,is_hammer_handle,is_hammer_bald,is_hammer
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-10-03 16:00:00,5.1085,5.1560,5.0897,5.1506,15222,0.008340,13463.0,8.653781,0.0663,0.013026,0.0421,0.039654,0.028055,0.039062,0.032588,0.0054,0.0188,-0.0134,-0.712766,0.004518,0.014962,0.000075,0.000075,0,1,1,0,1,1,1,1,1
2023-10-03 20:00:00,5.1506,5.1793,5.1457,5.1740,6339,0.004543,-8883.0,0.416437,0.0336,0.006530,0.0234,0.048616,0.038861,0.048173,0.041685,0.0053,0.0049,0.0004,0.081633,0.004493,0.014546,0.000073,0.000073,0,1,1,0,0,1,1,1,1
2023-10-04 12:00:00,5.1611,5.1611,5.1424,5.1486,2076,-0.004909,-4263.0,0.327496,0.0187,0.003636,-0.0125,0.048845,0.042881,0.049377,0.047559,0.0000,0.0062,-0.0062,-1.000000,0.004243,0.018397,0.000092,0.000092,1,0,1,1,0,0,1,1,0
2023-10-04 16:00:00,5.1488,5.1964,5.1430,5.1537,15178,0.000991,13102.0,7.311175,0.0534,0.010383,0.0049,0.048967,0.044829,0.054422,0.048727,0.0427,0.0058,0.0369,6.362069,0.010953,0.018657,0.000093,0.000093,0,1,1,0,1,0,1,1,0
2023-10-05 12:00:00,5.1946,5.2010,5.1801,5.1828,2293,0.002301,-3279.0,0.411522,0.0209,0.004035,-0.0118,0.052170,0.051152,0.056949,0.054614,0.0064,0.0027,0.0037,1.370370,0.010031,0.019418,0.000097,0.000097,1,0,1,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26 16:00:00,5.4926,5.5194,5.4843,5.4982,11949,0.000983,9955.0,5.992477,0.0351,0.006400,0.0056,0.241602,0.238439,0.244295,0.241225,0.0212,0.0083,0.0129,1.554217,0.005652,0.033099,0.000165,0.000165,0,1,1,1,0,0,1,1,0
2024-08-26 20:00:00,5.4977,5.4978,5.4809,5.4976,5598,-0.000109,-6351.0,0.468491,0.0169,0.003083,-0.0001,0.241804,0.238640,0.244466,0.241431,0.0001,0.0167,-0.0166,-0.994012,0.005663,0.030727,0.000154,0.000154,0,0,1,1,0,0,1,1,0
2024-08-27 12:00:00,5.5003,5.5111,5.4882,5.4953,1981,-0.000418,-3617.0,0.353876,0.0229,0.004173,-0.0050,0.241999,0.238855,0.244664,0.241641,0.0108,0.0071,0.0037,0.521127,0.005660,0.028032,0.000140,0.000140,1,0,1,0,1,0,1,1,0


In [17]:
ticker.candlestick_chart('is_hammer', start_date='2024-06-01')

In [ ]:
results = ticker.summarize_results(backtest)

In [ ]:
results

In [ ]:
ticker.candlestick_chart(key='is_hammer', plot_type='pattern')

In [ ]:
ticker.features.is_hammer.value_counts()